# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
# Function to filter rows where start/stop range overlap with the specified range
def filter_overlapping_ranges(frame_ranges, start=0, stop=np.inf):
    # Check for overlap
    overlaps = np.logical_or(np.logical_and(frame_ranges[:,0] < stop, frame_ranges[:,0] >= start),
                             np.logical_and(frame_ranges[:,1] <= stop, frame_ranges[:,1] > start))
    return frame_ranges[overlaps]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_long_comp"

In [10]:
TRODES_METADATA_DF = pd.read_pickle("./proc/{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX))

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
TRODES_METADATA_DF = TRODES_METADATA_DF.drop(columns=["current_subject", "video_timestamps", "recording", "session_path"], errors="ignore")
TRODES_METADATA_DF = TRODES_METADATA_DF.drop_duplicates(subset=["video_name"])

In [13]:
exploded_TRODES_METADATA_DF = TRODES_METADATA_DF.explode(column = ["tone_frames", "tone_timestamps"])

In [14]:
exploded_TRODES_METADATA_DF

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240317_151922_long_comp_subj_3-1_and_3-3,"[1, 88]","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,"[0, 87540]","[[0, 87540], [259142, 277740], [309743, 364544...",[]
0,20240317_151922_long_comp_subj_3-1_and_3-3,"[1286, 1486]","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,"[1287955, 1487958]","[[0, 87540], [259142, 277740], [309743, 364544...",[]
0,20240317_151922_long_comp_subj_3-1_and_3-3,"[3282, 3482]","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,"[3287980, 3487980]","[[0, 87540], [259142, 277740], [309743, 364544...",[]
0,20240317_151922_long_comp_subj_3-1_and_3-3,"[4480, 4679]","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,"[4487995, 4687995]","[[0, 87540], [259142, 277740], [309743, 364544...",[]
0,20240317_151922_long_comp_subj_3-1_and_3-3,"[5478, 5678]","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,"[5488007, 5688010]","[[0, 87540], [259142, 277740], [309743, 364544...",[]
...,...,...,...,...,...,...,...,...,...,...,...
18,20240321_114851_long_comp_subj_5-2_and_5-3,"[49514, 49713]","[[0, 104], [149, 153], [407, 413], [702, 705],...",[],20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,"[49606250, 49806253]","[[0, 105240], [148443, 152643], [408444, 41324...",[]
18,20240321_114851_long_comp_subj_5-2_and_5-3,"[50511, 50711]","[[0, 104], [149, 153], [407, 413], [702, 705],...",[],20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,"[50606263, 50806265]","[[0, 105240], [148443, 152643], [408444, 41324...",[]
18,20240321_114851_long_comp_subj_5-2_and_5-3,"[51909, 52108]","[[0, 104], [149, 153], [407, 413], [702, 705],...",[],20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,"[52006282, 52206280]","[[0, 105240], [148443, 152643], [408444, 41324...",[]
18,20240321_114851_long_comp_subj_5-2_and_5-3,"[54104, 54304]","[[0, 104], [149, 153], [407, 413], [702, 705],...",[],20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,"[54206307, 54406307]","[[0, 105240], [148443, 152643], [408444, 41324...",[]


In [15]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: np.array(x))


box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [16]:
columns_to_update = ["box_1_port_entry_frames", "box_2_port_entry_frames", "box_1_port_entry_timestamps", "box_2_port_entry_timestamps"]
tone_columns = ["tone_frames", "tone_frames", "tone_timestamps", "tone_timestamps"]

for col, tone_col in zip(columns_to_update, tone_columns):
    exploded_TRODES_METADATA_DF[col] = exploded_TRODES_METADATA_DF.apply(lambda x: filter_overlapping_ranges(x[col], x[tone_col][0], x[tone_col][1]), axis=1)

In [17]:
exploded_TRODES_METADATA_DF["tone_start_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[1])

In [18]:
exploded_TRODES_METADATA_DF["tone_start_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[1])


In [19]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF.drop(columns=["tone_frames", "tone_timestamps"], errors="ignore")

- Create a new column for start and stop

In [20]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF[sorted(exploded_TRODES_METADATA_DF.columns, key=lambda s: s.split('_')[-1])].copy()

In [21]:
exploded_TRODES_METADATA_DF.to_pickle("./proc/{}_per_video_trial_labels.pkl".format(OUTPUT_PREFIX))

- Exporting to CSV

In [22]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: ' '.join(map(str, x)))

box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [23]:
exploded_TRODES_METADATA_DF.head()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240317_151922_long_comp_subj_3-1_and_3-3,1,88,[ 1 88],,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,0,87540,[ 0 87540],
0,20240317_151922_long_comp_subj_3-1_and_3-3,1286,1486,[1300 1315] [1371 1380] [1420 1513],,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,1287955,1487958,[1301956 1317753] [1373156 1382756] [1422554 1...,
0,20240317_151922_long_comp_subj_3-1_and_3-3,3282,3482,[3277 3298] [3310 3356] [3357 3368] [3377 3378...,,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,3287980,3487980,[3281780 3302780] [3315978 3361781] [3362381 3...,
0,20240317_151922_long_comp_subj_3-1_and_3-3,4480,4679,[4383 4502] [4527 4851],,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,4487995,4687995,[4390791 4509995] [4535398 4859200],
0,20240317_151922_long_comp_subj_3-1_and_3-3,5478,5678,[5588 5778],,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"[3.1, 3.3]",2415571,48236575,5488007,5688010,[5597806 5788609],


In [24]:
exploded_TRODES_METADATA_DF.tail()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
18,20240321_114851_long_comp_subj_5-2_and_5-3,49514,49713,[49511 49526] [49533 49536] [49539 49545] [495...,,20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,49606250,49806253,[49603850 49619248] [49626050 49629248] [49631...,
18,20240321_114851_long_comp_subj_5-2_and_5-3,50511,50711,[50507 50519] [50520 50522] [50522 50523] [505...,,20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,50606263,50806265,[50601460 50613860] [50615060 50616265] [50617...,
18,20240321_114851_long_comp_subj_5-2_and_5-3,51909,52108,[51901 51925] [51926 51927] [51929 51947] [519...,,20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,52006282,52206280,[51999079 52022682] [52023677 52024677] [52026...,
18,20240321_114851_long_comp_subj_5-2_and_5-3,54104,54304,[54110 54113] [54113 54114] [54116 54120] [541...,,20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,54206307,54406307,[54212307 54213905] [54214309 54215504] [54217...,
18,20240321_114851_long_comp_subj_5-2_and_5-3,55203,55402,[55214 55224] [55225 55232] [55233 55239] [552...,,20240321_114851_long_comp_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",2981006,64420759,55306318,55506320,[55318118 55327323] [55328518 55335921] [55336...,


In [25]:
exploded_TRODES_METADATA_DF.to_csv("./proc/{}_per_video_trial_labels.csv".format(OUTPUT_PREFIX))